# Setup Code for VM related items
## !!!!When Saving, please ping revision so its saved in revision history!!!
Probs need to run anything under this section before code


In [1]:
!pip install alpaca-trade-api

     |████████████████████████████████| 81kB 6.2MB/s 
     |████████████████████████████████| 204kB 11.5MB/s 


Pandas Technical Analysis (Pandas TA) is an easy to use library that leverages the Pandas library with more than 120 Indicators and Utility functions. 

INFO HERE : https://github.com/twopirllc/pandas-ta#installation

USEFUL EXAMPLE: https://github.com/twopirllc/pandas-ta/blob/master/examples/AIExample.ipynb

In [2]:
!pip install --upgrade ta

  Created wheel for ta: filename=ta-0.7.0-cp36-none-any.whl size=28718 sha256=a3027e61f806e65ace65ca6d0957e0c1508c30ab0938f91b85b61fc82d777e06
  Stored in directory: /root/.cache/pip/wheels/dd/88/30/de9553fb54a474eb7480b937cdbb140bdda613d29cf4da7994
Successfully built ta


#Code
code cells are unfortunately not linked together and a such all code for program has to go on one cell

Alpaca Secret : Y01uwQ4EFeZQzanLj1UVs7NqyCAHvbwo6UyrxqFw

Alpaca Key : PK2K1WCVGJBQ0T4BKDIW

Yahoo Finance Key : 75ee96eb2fmsha6dbdb2310728dfp1301d2jsnbf21f7a615c1

In [15]:
import alpaca_trade_api as alpacaapi
import pandas as pd
import requests
import json
from ta import add_all_ta_features
from ta.utils import dropna
from ta.momentum import RSIIndicator
from ta.trend import MACD, SMAIndicator, EMAIndicator
from ta.momentum import StochasticOscillator
from ta.volume import OnBalanceVolumeIndicator


api = alpacaapi.REST("PK2K1WCVGJBQ0T4BKDIW", "Y01uwQ4EFeZQzanLj1UVs7NqyCAHvbwo6UyrxqFw", "https://paper-api.alpaca.markets")

################
# getStockData #
################

# https://alpaca.markets/docs/api-documentation/api-v2/market-data/bars/
# Check the above link for specific usage if you need to use it in your function
def getStockData(tick, length, frame):
  barset = api.get_barset(tick, frame , limit=length).df
  return barset

#Example usage
#  This will get the info for Tesla for the last 100 minutes
#  getStockData("TSLA", 100, "minute")


#Chris

#####################
# getStockDataMulti #
#####################

# getStockDataMulti creates a list of dataframes containing
# time-aggregated price and volume data 
def getStockDataMulti(ticks, length, frame):
  dataframes = []
  for tick in ticks:
    df = getStockData(tick, length, frame)
    dataframes.append({ tick: df })
  return dataframes

# Example usage
#   stocks = ["TSLA", "AAPL", "SNAP"]
#   data = getStockDataMulti(stocks, 5, "minute")
#   print(data)

######################
# getTrendingTickers #
######################

# getTrendingTickers creates a list of the current top 20
# trending stocks from Yahoo Finance
def getTrendingTickers():
  # build request
  url = "https://apidojo-yahoo-finance-v1.p.rapidapi.com/market/get-trending-tickers"
  querystring = {"region":"US"}
  headers = {
      'x-rapidapi-key': "75ee96eb2fmsha6dbdb2310728dfp1301d2jsnbf21f7a615c1",
      'x-rapidapi-host': "apidojo-yahoo-finance-v1.p.rapidapi.com"
  }
  # execute request
  response = requests.request("GET", url, headers=headers, params=querystring)
  raw_data = json.loads(response.text)['finance']['result']
  # extract symbols
  symbols = []
  for quote in raw_data[0]['quotes']:
    symbols.append(quote['symbol'])
  return symbols

# Example usage
#   symbols = getTrendingTickers()
#   print(symbols)



#Collin

#############
# sellLimit #
#############

# A sell limit is submitted as a gtc(Good 'Til Cancelled) time in force which 
# keeps the sell limit order open until it is fulfilled or cancelled.
# You must pass in the current price of the tick as the third argument when the
# sell limit is placed. This is so the 10% price increase can be calculated.
def sellLimit(tick, quantity, current_price):
    try:
        api.submit_order(
            symbol=symbol,
            qty=quantity,
            side='sell',
            type='limit',
            time_in_force='gtc',
            limit_price = current_price * 1.1
        )
    # if an excpetion is thrown, prints exception message and returns 0 to 
    # indicate a failed sell limit placement
    except Exception as e:
        print(e)
        return 0

    # sellLimit function returns 1 if the order is placed succesfully
    else:
        return 1
        
# Example usage
#   result = sellLimit('AAPL', 1, 445)
#   print("Result of sell limit placement is 0 for fail, 1 for success: ", result)








#Ali

def sellStocks(stockName, quantity, sellType, ToF):
  return api.submit_order(
      symbol=stockName,
      qty=quantity,
      side='sell',
      type=sellType, # market, limit, stop, stop_limit, or trailing_stop
      time_in_force=ToF #day, gtc, opg, cls, ioc, fok.
  )

# Example usage
stockName = 'AAPL'
quantity = 5
sellType = 'market'
ToF = 'gtc'
#stockOrder = sellStocks(stockName, quantity, sellType, ToF)

#print(stockOrder)










#Ben

#####################
#     buyStocks     #
#####################
def buyStocks(stockName, quantity, buyType, ToF):
  return api.submit_order(
      symbol=stockName,
      qty=quantity,
      side='buy',
      type=buyType, # market, limit, stop, stop_limit, or trailing_stop
      time_in_force=ToF #day, gtc, opg, cls, ioc, fok.
  )

# Example usage
stockName = 'AAPL'
quantity = 5
buyType = 'market'
ToF = 'gtc'
#stockOrder = buyStocks(stockName, quantity, buyType, ToF)

#print(stockOrder)








#Bryan

def stopLimit(tick, quantity, current_price):
    try:
        api.submit_order(
            symbol=symbol,
            qty=quantity,
            side='stop',
            type='stop_limit',
            time_in_force='gtc',
            limit_price = current_price * 0.7
        )
    # if an excpetion is thrown, prints exception message and returns 0 to 
    # indicate a failed sell limit placement
    except Exception as e:
        print(e)
        return 0

    # stopLimit function returns 1 if the order is placed succesfully
    else:
        return 1


#determines if the AI should buy right now
def isBuy(ticker, duration, timeFrame):
  df = getStockData(ticker, duration, timeFrame)

  #Drop Null Values
  df = dropna(df)


  #Declare Indicators
  rsi = RSIIndicator(close=df[ticker]["close"], window=20, fillna=False)
  macd = MACD(close=df[ticker]["close"], window_slow=20, window_fast = 16, window_sign= 7, fillna = False)
  stoch = StochasticOscillator(high=df[ticker]["high"],low=df[ticker]["low"],close=df[ticker]["close"], window=14, smooth_window=3, fillna=False)
  onvol = OnBalanceVolumeIndicator(close=df[ticker]["close"], volume=df[ticker]["volume"])
  ema = EMAIndicator(close=df[ticker]["close"], window=200, fillna=False)
  sma = SMAIndicator(close=df[ticker]["close"], window=50, fillna=False)

  # Add Technical Analysis indicators to dataframes
  df['rsi'] = rsi.rsi()
  df['macd'] = macd.macd_diff()
  df['stoch'] = stoch.stoch_signal()
  df['onbalvol'] = onvol.on_balance_volume()
  df['ema'] = ema.ema_indicator()
  df['sma'] = sma.sma_indicator()

  # get todays entry and run filter on it
  curr = df.tail(1)
  curr = curr.loc[(curr["rsi"]>40) & (curr["rsi"]<55)]
  curr = curr.loc[(curr["macd"]> -1) & (curr["macd"]< 1)]
  curr = curr.loc[(curr['stoch']> 20) & (curr['stoch']< 40)]

  # if not empty after filter run, check close price is within 25% below or above ema, then check onbalance volume for slope
  if not curr.empty:
    if (curr["close"] > (curr["close"] - curr["ema"]*.25) or curr["close"] < (curr["close"] + curr["ema"]*.25)):
      sday = df.tail(7)
      head = sday.head(1) 
      tail = sday.tail(1)
      dayone = (head.index.values[0] - pd.Timestamp("1970-01-01")) // pd.Timedelta("1s")
      dayseven = (tail.index.values[0] - pd.Timestamp("1970-01-01")) // pd.Timedelta("1s")
      slope = (int(tail["onbalvol"].values) - int(head["onbalvol"].values)) / (dayseven - dayone)
      # positive returns true, negative returns false
      if slope > 0:
        return true
      else:
        return false
    #if price not within 25% of ema, return false
    else:
      return false
  #if dataframe is null, return false
  else:
    return false


#determines if the AI should buy right now
def isSell(ticker, duration, timeFrame):
  df = getStockData(ticker, duration, timeFrame)

  #Drop Null Values
  df = dropna(df)


  #Declare Indicators
  rsi = RSIIndicator(close=df[ticker]["close"], window=20, fillna=False)
  macd = MACD(close=df[ticker]["close"], window_slow=20, window_fast = 16, window_sign= 7, fillna = False)
  stoch = StochasticOscillator(high=df[ticker]["high"],low=df[ticker]["low"],close=df[ticker]["close"], window=14, smooth_window=3, fillna=False)
  onvol = OnBalanceVolumeIndicator(close=df[ticker]["close"], volume=df[ticker]["volume"])
  ema = EMAIndicator(close=df[ticker]["close"], window=200, fillna=False)
  sma = SMAIndicator(close=df[ticker]["close"], window=50, fillna=False)

  # Add Technical Analysis indicators to dataframes
  df['rsi'] = rsi.rsi()
  df['macd'] = macd.macd_diff()
  df['stoch'] = stoch.stoch_signal()
  df['onbalvol'] = onvol.on_balance_volume()
  df['ema'] = ema.ema_indicator()
  df['sma'] = sma.sma_indicator()

  # get todays entry and run filter on it
  curr = df.tail(1)
  curr = curr.loc[(curr["rsi"]>65) & (curr["rsi"]<90)]
  curr = curr.loc[(curr["macd"]> -1) & (curr["macd"]< 0)]
  curr = curr.loc[(curr['stoch']> 70) & (curr['stoch']< 100)]

  # if not empty after filter run, check close price is within 25% below or above ema, then check onbalance volume for slope
  if not curr.empty():
    if (curr["close"] > (curr["close"] - curr["ema"]*.25) or curr["close"] < (curr["close"] + curr["ema"]*.25)):
      sday = df.tail(7)
      head = sday.head(1) 
      tail = sday.tail(1)
      dayone = (head.index.values[0] - pd.Timestamp("1970-01-01")) // pd.Timedelta("1s")
      dayseven = (tail.index.values[0] - pd.Timestamp("1970-01-01")) // pd.Timedelta("1s")
      slope = (int(tail["onbalvol"].values) - int(head["onbalvol"].values)) / (dayseven - dayone)
      # negative slope returns true, positive returns false
      if slope < 0:
        return true
      else:
        return false
    #if price not within 25% of ema, return false
    else:
      return false
  #if dataframe is null, return false
  else:
    return false




###########
# ML CODE #
###########
df = getStockData("TSLA", 100, "day")
ef = getStockData("TSLA", 400, "day")

df = dropna(df)

# Initialize Bollinger Bands Indicator
rsi = RSIIndicator(close=df["TSLA"]["close"], window=20, fillna=False)
macd = MACD(close=df["TSLA"]["close"], window_slow=20, window_fast = 16, window_sign= 7, fillna = False)
stoch = StochasticOscillator(high=df["TSLA"]["high"],low=df["TSLA"]["low"],close=df["TSLA"]["close"], window=14, smooth_window=3, fillna=False)
onvol = OnBalanceVolumeIndicator(close=df["TSLA"]["close"], volume=df["TSLA"]["volume"])
ema = EMAIndicator(close=ef["TSLA"]["close"], window=200, fillna=False)
sma = SMAIndicator(close=ef["TSLA"]["close"], window=50, fillna=False)

# Add Technical Analysis indicators
df['rsi'] = rsi.rsi()
df['macd'] = macd.macd_diff()
df['stoch'] = stoch.stoch_signal()
df['onbalvol'] = onvol.on_balance_volume()
df['ema'] = ema.ema_indicator()
df['sma'] = sma.sma_indicator()

pd.set_option('display.max_columns', None)


is_bear = df.loc[(df["rsi"]>65) & (df["rsi"]<90)]
is_bear = is_bear[(is_bear["macd"] > -.5) & (is_bear["macd"] < 0)]
is_bear = is_bear[(is_bear["stoch"] > 70) & (is_bear["stoch"]< 100)]
is_bull = df.loc[(df["rsi"]>40) & (df["rsi"]<55)]
is_bull = is_bull.loc[(is_bull["macd"]> -1) & (is_bull["macd"]< 1)]
is_bull = is_bull.loc[(is_bull['stoch']> 20) & (is_bull['stoch']< 40)]


sday = df.tail(7)
head = sday.head(1) 
tail = sday.tail(1)
dayone = (head.index.values[0] - pd.Timestamp("1970-01-01")) // pd.Timedelta("1s")
dayseven = (tail.index.values[0] - pd.Timestamp("1970-01-01")) // pd.Timedelta("1s")
slope = (int(tail["onbalvol"].values) - int(head["onbalvol"].values)) / (dayseven - dayone)



#print( "YDIFF: " + str((int(tail["onbalvol"].values) - int(head["onbalvol"].values))))
#print( "XDIFF: " + str((dayseven - dayone)))
#On Balance Value slope which proves a bullish volume trend or a bearish volume trend ############### Near positive to negative crossover is a bearish trend and vice versa is a bullish

print("########################BUY#############################")
print(is_bull)
print("########################SELL#############################")
print(is_bear)



########################BUY#############################
                             TSLA                                     \
                             open    high     low    close    volume   
time                                                                   
2020-10-27 00:00:00-04:00  423.76  430.50  420.10  424.471  21007688   
2020-11-03 00:00:00-05:00  409.73  427.77  406.69  423.740  32428696   

                                 rsi      macd      stoch   onbalvol  \
                                                                       
time                                                                   
2020-10-27 00:00:00-04:00  49.154280 -0.575185  25.194910  270726220   
2020-11-03 00:00:00-05:00  50.549945 -0.587115  30.854419  287183283   

                                  ema         sma  
                                                   
time                                               
2020-10-27 00:00:00-04:00  786.904209  716.680092  
2020-11-03 00